In [1]:
import os
import shutil
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [2]:
image_path1 = r'.\data\HAM10000_images_part_1'
image_path2 = r'.\data\HAM10000_images_part_2'

# Directory for the final combined images
final_image_dataset = './data/skin-cancer-mnist-ham10000_combined'

In [3]:
%cd ../

C:\Users\Sarthak\Documents\GitHub\skincancer


In [4]:
metadata_path = './data/HAM10000_metadata.csv'
meta_data = pd.read_csv(metadata_path)

# Add a new column for the image paths in the combined folder
meta_data['Image_path'] = meta_data['image_id'].apply(lambda x: os.path.join(final_image_dataset, f"{x}.jpg"))

# Initialize and fit LabelEncoder
le = LabelEncoder()
meta_data['label'] = le.fit_transform(meta_data["dx"])

# Convert labels to string format
meta_data['label'] = meta_data['label'].astype(str)

# Resample to balance the dataset
n_samples = 4500 # Maximum count of the 'nv' class

# Separate each class
df_nv = meta_data[meta_data['dx'] == 'nv']
df_mel = meta_data[meta_data['dx'] == 'mel']
df_bkl = meta_data[meta_data['dx'] == 'bkl']
df_bcc = meta_data[meta_data['dx'] == 'bcc']
df_akiec = meta_data[meta_data['dx'] == 'akiec']
df_vasc = meta_data[meta_data['dx'] == 'vasc']
df_df = meta_data[meta_data['dx'] == 'df']

In [5]:
df_nv_balanced = resample(df_nv, replace=False, n_samples=n_samples, random_state=42)
df_mel_balanced = resample(df_mel, replace=True, n_samples=n_samples, random_state=42)
df_bkl_balanced = resample(df_bkl, replace=True, n_samples=n_samples, random_state=42)
df_bcc_balanced = resample(df_bcc, replace=True, n_samples=n_samples, random_state=42)
df_akiec_balanced = resample(df_akiec, replace=True, n_samples=n_samples, random_state=42)
df_vasc_balanced = resample(df_vasc, replace=True, n_samples=n_samples, random_state=42)
df_df_balanced = resample(df_df, replace=True, n_samples=n_samples, random_state=42)

In [6]:
balanced_meta_data = pd.concat([df_nv_balanced, df_mel_balanced, df_bkl_balanced,
                        df_bcc_balanced, df_akiec_balanced, df_vasc_balanced,
                        df_df_balanced])
# Mapping labels to their full names
label_mapping = {
'nv': 'Melanocytic Nevus',
'mel': 'Melanoma',
'bkl': 'Benign Keratosis',
'bcc': 'Basal Cell Carcinoma',
'akiec': 'Actinic Keratosis',
'vasc': 'Vascular Lesion',
'df': 'Dermatofibroma'
}

# Replace the labels with full names
balanced_meta_data['dx'] = balanced_meta_data['dx'].replace(label_mapping)

# Now, balanced_meta_data['dx'] will contain the full names
print(balanced_meta_data['dx'].value_counts())

# Shuffle the combined dataset
balanced_meta_data = balanced_meta_data.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"Balanced dataset class distribution:\n{balanced_meta_data['dx'].value_counts()}")

dx
Melanocytic Nevus       4500
Melanoma                4500
Benign Keratosis        4500
Basal Cell Carcinoma    4500
Actinic Keratosis       4500
Vascular Lesion         4500
Dermatofibroma          4500
Name: count, dtype: int64
Balanced dataset class distribution:
dx
Basal Cell Carcinoma    4500
Melanocytic Nevus       4500
Actinic Keratosis       4500
Dermatofibroma          4500
Melanoma                4500
Vascular Lesion         4500
Benign Keratosis        4500
Name: count, dtype: int64


In [7]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Perform initial train-test split without stratification
train_meta, test_meta = train_test_split(balanced_meta_data, test_size=0.3, random_state=42)
train_meta, val_meta = train_test_split(train_meta, test_size=0.3, random_state=42)

print(f"Training set size: {len(train_meta)}")
print(f"Validation set size: {len(val_meta)}")
print(f"Testing set size: {len(test_meta)}")

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
rescale=1./255,
rotation_range=20,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True,
fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Train generator
train_generator = train_datagen.flow_from_dataframe(
dataframe=train_meta,
directory='.',
x_col='Image_path',
y_col='label',
target_size=(128, 128),
batch_size=32,
class_mode='categorical'
)

# Validation generator
val_generator = test_datagen.flow_from_dataframe(
dataframe=val_meta,
directory='.',
x_col='Image_path',
y_col='label',
target_size=(128,128),
batch_size=32,
class_mode='categorical',
shuffle=False
)

# Test generator
test_generator = test_datagen.flow_from_dataframe(
dataframe=test_meta,
directory='.',
x_col='Image_path',
y_col='label',
target_size=(128, 128),
batch_size=32,
class_mode='categorical',
shuffle=False
)

Training set size: 15435
Validation set size: 6615
Testing set size: 9450
Found 15435 validated image filenames belonging to 7 classes.
Found 6615 validated image filenames belonging to 7 classes.
Found 9450 validated image filenames belonging to 7 classes.


In [14]:
base_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 63, 63, 32)           864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 63, 63, 32)           96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 63, 63, 32)           0         ['batch_normalizati

In [15]:
base_model = tf.keras.applications.InceptionV3(input_shape=(128,128,3),
                                               include_top=False,
                                               weights='imagenet')

In [20]:
# Set all layers to non-trainable first
for layer in base_model.layers:
    layer.trainable = False

# Make the last 10 layers trainable
for layer in base_model.layers[-17:]:
    layer.trainable = True


In [21]:
base_model.summary()


Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 conv2d_94 (Conv2D)          (None, 63, 63, 32)           864       ['input_2[0][0]']             
                                                                                                  
 batch_normalization_94 (Ba  (None, 63, 63, 32)           96        ['conv2d_94[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_94 (Activation)  (None, 63, 63, 32)           0         ['batch_normalizati

In [22]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
hidden_dense_layer_1 = tf.keras.layers.Dense(2048, activation='relu')
prediction_layer = tf.keras.layers.Dense(7, activation='softmax')

In [23]:
inputs = tf.keras.Input(shape=(128,128,3))
x = base_model(inputs, training=False)
x = global_average_layer(x)
x = hidden_dense_layer_1(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [24]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 inception_v3 (Functional)   (None, 2, 2, 2048)        21802784  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 2048)              4196352   
                                                                 
 dense_1 (Dense)             (None, 7)                 14343     
                                                                 
Total params: 26013479 (99.23 MB)
Trainable params: 5261319 (20.07 MB)
Non-trainable params: 20752160 (79.16 MB)
______________

In [25]:
callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                                            verbose =1,
                                            mode ='max',
                                            baseline=0.65,
                                            restore_best_weights=True,
                                            patience=15)

In [35]:
history = model.fit(
  train_generator,
  validation_data=val_generator,
  callbacks=[callbacks],
  epochs=100
)

Epoch 1/100
483/483 [==============================] - 269s 557ms/step - loss: 0.5349 - accuracy: 0.8010 - val_loss: 0.5289 - val_accuracy: 0.8142
Epoch 2/100
483/483 [==============================] - 265s 548ms/step - loss: 0.5303 - accuracy: 0.8045 - val_loss: 0.5267 - val_accuracy: 0.8098
Epoch 3/100
483/483 [==============================] - 266s 550ms/step - loss: 0.5281 - accuracy: 0.8041 - val_loss: 0.5575 - val_accuracy: 0.7962
Epoch 4/100
483/483 [==============================] - 266s 550ms/step - loss: 0.5177 - accuracy: 0.8054 - val_loss: 0.5544 - val_accuracy: 0.8086
Epoch 5/100
483/483 [==============================] - 266s 551ms/step - loss: 0.5247 - accuracy: 0.8045 - val_loss: 0.5104 - val_accuracy: 0.8121
Epoch 6/100
483/483 [==============================] - 268s 555ms/step - loss: 0.5198 - accuracy: 0.8082 - val_loss: 0.5481 - val_accuracy: 0.8033
Epoch 7/100
483/483 [==============================] - 266s 550ms/step - loss: 0.5279 - accuracy: 0.8062 - val_loss: 0

In [36]:
model_version = max([int(i) for i in os.listdir("./models") if i.isdigit()] + [0]) + 1

# model.save(f"./models/{model_version}.h5")

In [40]:
model_version=3

In [42]:
model.save(f"./models/{model_version}_inceptionV3")

INFO:tensorflow:Assets written to: ./models/3_inceptionV3\assets


INFO:tensorflow:Assets written to: ./models/3_inceptionV3\assets


In [39]:
history = model.fit(
  train_generator,
  validation_data=val_generator,
  callbacks=[callbacks],
  epochs=50
)

Epoch 1/50
483/483 [==============================] - 273s 565ms/step - loss: 0.4026 - accuracy: 0.8561 - val_loss: 0.4008 - val_accuracy: 0.8701
Epoch 2/50
483/483 [==============================] - 274s 567ms/step - loss: 0.3889 - accuracy: 0.8604 - val_loss: 0.4056 - val_accuracy: 0.8692
Epoch 3/50
483/483 [==============================] - 274s 568ms/step - loss: 0.3841 - accuracy: 0.8629 - val_loss: 0.4186 - val_accuracy: 0.8565
Epoch 4/50
483/483 [==============================] - 274s 568ms/step - loss: 0.3931 - accuracy: 0.8568 - val_loss: 0.4141 - val_accuracy: 0.8571
Epoch 5/50
483/483 [==============================] - 274s 567ms/step - loss: 0.3923 - accuracy: 0.8581 - val_loss: 0.4177 - val_accuracy: 0.8547
Epoch 6/50
483/483 [==============================] - 274s 567ms/step - loss: 0.3946 - accuracy: 0.8582 - val_loss: 0.4167 - val_accuracy: 0.8577
Epoch 7/50
483/483 [==============================] - 274s 567ms/step - loss: 0.3942 - accuracy: 0.8578 - val_loss: 0.4200 -